In [1]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [2]:
# load data
 
df=pd.read_csv('Waveform_withoutdupl_norm_v01.csv')  
del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)
df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att13,att14,att15,att16,att17,att18,att19,att20,att21,outlier
0,0.323657,0.793785,0.848661,0.660063,0.622244,0.755102,0.760762,0.671655,0.603723,0.636170,...,0.281899,0.203557,0.286607,0.474383,0.352483,0.358857,0.275626,0.465937,0.220884,1
1,0.423803,0.769774,0.632130,0.572471,0.747495,0.670408,0.715232,0.617958,0.637411,0.457447,...,0.302671,0.375494,0.296429,0.383302,0.545278,0.394286,0.451025,0.470803,0.468541,1
2,0.477504,0.580508,0.554133,0.476538,0.851703,0.509184,0.745861,0.785211,0.508865,0.519149,...,0.341246,0.384387,0.229464,0.350095,0.117819,0.406857,0.149203,0.402676,0.370817,1
3,0.645864,0.570621,0.525029,0.631908,0.682365,0.382653,0.478477,0.608275,0.476950,0.190426,...,0.500495,0.555336,0.463393,0.537951,0.564752,0.485714,0.576310,0.518248,0.597055,1
4,0.219158,0.690678,0.530850,0.715328,0.687375,0.571429,0.649007,0.562500,0.523050,0.303191,...,0.190900,0.408103,0.322321,0.425996,0.400195,0.701714,0.515945,0.294404,0.463186,1


In [3]:
# data to values
df = df.values

In [4]:
# Deep ExtraTrees
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/waveform.json")  
gc = GCForest(config)

In [16]:
# train test
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [6]:
# count class
len(np.unique(y_train))

2

In [7]:
  # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
  

[ 2018-04-25 10:15:49,579][cascade_classifier.fit_transform] X_groups_train.shape=[(2410, 21)],y_train.shape=(2410,),X_groups_test.shape=[(1033, 21)],y_test.shape=(1033,)
[ 2018-04-25 10:15:49,582][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-04-25 10:15:49,584][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 10:15:49,585][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-04-25 10:15:49,587][cascade_classifier.fit_transform] X_train.shape=(2410, 21),X_test.shape=(1033, 21)
[ 2018-04-25 10:15:49,589][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2410, 21), X_cur_test.shape=(1033, 21)
[ 2018-04-25 10:15:50,266][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=96.69%
[ 2018-04-25 10:15:51,053][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.10%
[ 2018-04-25 10:15:51,838][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [8]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtratTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 10:15:57,831][cascade_classifier.transform] X_groups_test.shape=[(1033, 21)]
[ 2018-04-25 10:15:57,834][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 10:15:57,836][cascade_classifier.transform] X_test.shape=(1033, 21)
[ 2018-04-25 10:15:57,838][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1033, 21)


Test Accuracy of ExtratTrees = 97.289448 %


In [9]:
#  predict_proba + Data 
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 10:15:59,350][cascade_classifier.transform] X_groups_test.shape=[(1033, 21)]
[ 2018-04-25 10:15:59,353][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 10:15:59,355][cascade_classifier.transform] X_test.shape=(1033, 21)
[ 2018-04-25 10:15:59,357][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1033, 21)


In [10]:
# New data
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [11]:
X_test_new.shape

(1033, 23)

In [12]:
X_train_new.shape

(2410, 23)

In [13]:
# Cascade ExtraTrees
config = load_json("./examples/waveformgc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 10:16:00,867][cascade_classifier.fit_transform] X_groups_train.shape=[(2410, 23)],y_train.shape=(2410,),X_groups_test.shape=[(1033, 23)],y_test.shape=(1033,)
[ 2018-04-25 10:16:00,869][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 10:16:00,870][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 10:16:00,871][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 10:16:00,873][cascade_classifier.fit_transform] X_train.shape=(2410, 23),X_test.shape=(1033, 23)
[ 2018-04-25 10:16:00,875][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2410, 23), X_cur_test.shape=(1033, 23)
[ 2018-04-25 10:16:01,581][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.52%
[ 2018-04-25 10:16:02,382][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.93%
[ 2018-04-25 10:16:03,200][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

[ 2018-04-25 10:16:34,295][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=97.93%
[ 2018-04-25 10:16:35,110][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=98.76%
[ 2018-04-25 10:16:35,895][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=97.93%
[ 2018-04-25 10:16:36,733][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=97.10%
[ 2018-04-25 10:16:37,548][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=98.76%
[ 2018-04-25 10:16:38,319][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.17%
[ 2018-04-25 10:16:39,084][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=97.51%
[ 2018-04-25 10:16:39,838][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.34%


In [14]:
# deep
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of Extratrees = {:.6f} %".format(acc * 100),i,'fois')

In [15]:
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/waveformgc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-04-25 10:16:49,078][cascade_classifier.fit_transform] X_groups_train.shape=[(2410, 23)],y_train.shape=(2410,),X_groups_test.shape=[(1033, 23)],y_test.shape=(1033,)
[ 2018-04-25 10:16:49,080][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 10:16:49,082][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 10:16:49,083][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 10:16:49,085][cascade_classifier.fit_transform] X_train.shape=(2410, 23),X_test.shape=(1033, 23)
[ 2018-04-25 10:16:49,087][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2410, 23), X_cur_test.shape=(1033, 23)
[ 2018-04-25 10:16:49,702][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.93%
[ 2018-04-25 10:16:50,481][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.51%
[ 2018-04-25 10:16:51,270][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

[ 2018-04-25 10:17:20,984][cascade_classifier.transform] group_dims=[23]
[ 2018-04-25 10:17:20,985][cascade_classifier.transform] X_test.shape=(1033, 23)
[ 2018-04-25 10:17:20,987][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1033, 23)
[ 2018-04-25 10:17:22,494][cascade_classifier.fit_transform] X_groups_train.shape=[(2410, 23)],y_train.shape=(2410,),X_groups_test.shape=[(1033, 23)],y_test.shape=(1033,)
[ 2018-04-25 10:17:22,496][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 10:17:22,498][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 10:17:22,499][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 10:17:22,501][cascade_classifier.fit_transform] X_train.shape=(2410, 23),X_test.shape=(1033, 23)
[ 2018-04-25 10:17:22,503][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2410, 23), X_cur_test.shape=(1033, 23)


('Test Accuracy of Extratrees = 97.870281 %', 0, 'fois')


[ 2018-04-25 10:17:23,125][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.11%
[ 2018-04-25 10:17:23,893][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.51%
[ 2018-04-25 10:17:24,716][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.93%
[ 2018-04-25 10:17:25,492][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.34%
[ 2018-04-25 10:17:26,309][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.51%
[ 2018-04-25 10:17:27,059][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.51%
[ 2018-04-25 10:17:27,821][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.34%
[ 2018-04-25 10:17:28,605][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.93%


[ 2018-04-25 10:18:00,270][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.76%
[ 2018-04-25 10:18:01,041][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=97.93%
[ 2018-04-25 10:18:01,826][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.34%
[ 2018-04-25 10:18:02,660][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=97.08%
[ 2018-04-25 10:18:02,814][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=98.09%
[ 2018-04-25 10:18:02,816][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=97.77%
[ 2018-04-25 10:18:02,818][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=98.09%
[ 2018-04-25 10:18:02,820][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=97.77%
[ 2018-04-25 10:18:02,823][

...

[ 2018-04-25 10:30:53,643][cascade_classifier.fit_transform] X_train.shape=(2410, 23),X_test.shape=(1033, 23)
[ 2018-04-25 10:30:53,645][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2410, 23), X_cur_test.shape=(1033, 23)


('Test Accuracy of Extratrees = 97.676670 %', 18, 'fois')


[ 2018-04-25 10:30:54,251][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.52%
[ 2018-04-25 10:30:55,031][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.10%
[ 2018-04-25 10:30:55,804][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.51%
[ 2018-04-25 10:30:56,570][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.34%
[ 2018-04-25 10:30:57,362][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.34%
[ 2018-04-25 10:30:58,190][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.51%
[ 2018-04-25 10:30:58,960][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.34%
[ 2018-04-25 10:30:59,775][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.34%


[ 2018-04-25 10:31:31,206][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.76%
[ 2018-04-25 10:31:32,047][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=97.51%
[ 2018-04-25 10:31:32,818][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.34%
[ 2018-04-25 10:31:33,623][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=97.08%
[ 2018-04-25 10:31:33,776][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=97.97%
[ 2018-04-25 10:31:33,778][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=97.77%
[ 2018-04-25 10:31:33,780][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=97.97%
[ 2018-04-25 10:31:33,782][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=97.77%
[ 2018-04-25 10:31:33,784][

('Test Accuracy of Extratrees = 97.773475 %', 19, 'fois')


# ('Test Accuracy of Extratrees = 97.870281 %', 0, 'fois')